**Scheduling Fairness**
- Each team should play the same number of games
- No unfair rest periods
- No unfair consecutive home/away games
- No unfair travel distances

**Competitive Balance**
- Match ups are between similarly ranked teams

**Geographic Fairness**
- Minimize total travel distance
- Minimize travel distance for each team
- Encourage "regional" match ups

**Mathematical Model**

Variables:
- $x_{ijt}$: $1$ if team $i$ plays team $j$ at time $t$, $0$ otherwise
- $d_{ij}$: distance between teams $i$ and $j$
- $h_{ij}$: $1$ if team $i$ plays at home against team $j$, $0$ otherwise

Objective Functions:
- Minimize total travel distance: $$D = \sum_{i,j,t} d_{ij} x_{ijt}$$
- Minimize rest disparity: $$R = max{i,j,t} |r_{it} - r_{jt}|$$

**Constraints**

Total number of games each team plays is fixed 
$$\sum_{j,t} x_{ijt} = n$$
No team can play two games at the same time
$$\sum_{i,j} x_{ijt} \leq 1$$

Each team should play the same number of games:
$2k$ where $k$ are home and $k$ are away.
The length of the season is between $L$ and $U$.
Teams cannot play everyday for more than $D$ days.

In [5]:
from ortools.sat.python import cp_model
import numpy as np
import itertools
import matplotlib.pyplot as plt

# Parameters
N = 20  # Number of teams
D = 180  # Number of days in a season
teams = list(range(N))
days = list(range(D))

ModuleNotFoundError: No module named 'ortools'

In [ ]:
import folium
from IPython.display import display

# Country Data
names = [
    "Argentina",
    "Brazil",
    "Colombia",
    "Uraguay",
    "Ecuador",
    "Mexico",
    "Panama",
    "Morocco",
    "Egypt",
    "Spain",
    "UK",
    "Italy",
    "France",
    "Germany",
    "Netherlands",
    "Japan",
    "South Korea",
    "Australia",
    "Turkey",
    "New Zealand",
]

elo_ratings = [
    2140,
    1994,
    1953,
    1922,
    1911,
    1817,
    1724,
    1807,
    1668,
    2150,
    2012,
    1914,
    2031,
    1988,
    1967,
    1875,
    1745,
    1736,
    1837,
    1596,
]

locations = [
    (-38.4161, -63.6167),
    (-14.2350, -51.9253),
    (4.5709, -74.2973),
    (-32.5228, -55.7658),
    (-1.8312, -78.1834),
    (23.6345, -102.5528),
    (8.5376, -80.7821),
    (21.1280, -13.1628),
    (26.8206, 30.8025),
    (40.4637, -3.7492),
    (55.3781, -3.4360),
    (41.8719, 12.5674),
    (46.6034, 1.8883),
    (51.1657, 10.4515),
    (52.1326, 5.2913),
    (36.2048, 138.2529),
    (35.9078, 127.7669),
    (-25.2744, 133.7751),
    (38.9637, 35.2433),
    (-40.9006, 174.8860),
]

# plot all countries on a map
m = folium.Map(location=[20, 20], zoom_start=2)
for name, rating, location in zip(names, elo_ratings, locations):
    folium.Marker(
        location=location,
        icon=folium.Icon(color="red"),
        tooltip=f"{name} [{rating}]",
    ).add_to(m)
display(m)

In [ ]:
from math import radians, sin, cos, sqrt, atan2


# DISTANCE CALCULATIONS
def haversine(pos1, pos2):
    lat1, lon1 = pos1
    lat2, lon2 = pos2
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371.0 # Radius of Earth in kilometers
    return R * c # Distance in kilometers


def estimate_flight_time(pos1, pos2, speed_kmh=900):
    distance = haversine(pos1, pos2)
    time_hours = distance / speed_kmh
    return time_hours

estimate_flight_time(locations[0], locations[1])  # Example flight time between Argentina and Brazil

3.2491632111878563

In [ ]:
# Fake coordinates for cities (latitude, longitude)
np.random.seed(0)

# Create model
model = cp_model.CpModel()

# x[i][t][j] = 1 if team i plays at home against team j in week t
x = {
    (i, t, j): model.NewBoolVar(f"x_{i}_{t}_{j}")
    for i, j in itertools.permutations(teams, 2)
    for t in days
}

# Constraints

# # 1. Each team plays at most per day (either home or away)
# for t in days:
#     for team in teams:
#         model.Add(
#             sum(x[team, t, opp] for opp in teams if opp != team)
#             + sum(x[opp, t, team] for opp in teams if opp != team)
#             <= 1
#         )

# 2. Each pair of teams plays at most once (i vs j or j vs i over the entire season)
for i, j in itertools.combinations(teams, 2):
    model.Add(sum(x[i, t, j] + x[j, t, i] for t in days) <= 1)

# 3. Each team plays exactly g home and g away games, 2g games in total
half_games = 4
for team in teams:
    model.Add(
        sum(x[team, t, opp] for t in days for opp in teams if opp != team) == half_games
    )
    model.Add(
        sum(x[opp, t, team] for t in days for opp in teams if opp != team) == half_games
    )

# 4. Each team must have minimum break between games of 3 days (e.g., no back-to-back games)
# so at most one game every 3 days. NOTE: This replaces #1
for team in teams:
    for t in range(D - 3):
        model.Add(
            sum(
                x[team, t + d, opp] + x[opp, t + d, team]
                for d in range(4)
                for opp in teams
                if opp != team
            )
            <= 1
        )

# 5. Avoid Insane timezone Travel

# Objective: minimize total travel distance for away teams
total_travel = sum(
    x[(j, t, i)] * estimate_flight_time(locations[i], locations[j])
    for i in teams
    for t in days
    for j in teams
    if i != j
)

# Objective: minimize elo deviation between a team and the teams they play against.
elo_deviation = sum(
    (elo_ratings[i] - elo_ratings[j]) ** 2 / elo_ratings[i] * x[(i, t, j)] 
    for i in teams
    for t in days
    for j in teams
    if i != j
)

# Objective: Avoid conflict with other major events
# (e.g., holidays, playoffs, etc.)

cost = total_travel + elo_deviation

model.Minimize(cost)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Optimized Weekly Schedule (Home vs Away):\n")
    for t in days:
        print(f"Day {t+1}:")
        for i, j in itertools.permutations(teams, 2):
            if solver.Value(x[i, t, j]):
                print(f"  {names[i]} (home) vs {names[j]} (away)")
    print(f"\nTotal travel time: {solver.ObjectiveValue():.2f} hrs")
else:
    print("No feasible solution found.")


Optimized Weekly Schedule (Home vs Away):

Day 1:
  New Zealand (home) vs Australia (away)
Day 2:
Day 3:
Day 4:
Day 5:
Day 6:
Day 7:
Day 8:
Day 9:
  Argentina (home) vs Spain (away)
Day 10:
  Brazil (home) vs Ecuador (away)
  Japan (home) vs Australia (away)
Day 11:
Day 12:
Day 13:
  Panama (home) vs Morocco (away)
  South Korea (home) vs Turkey (away)
Day 14:
  France (home) vs Brazil (away)
Day 15:
Day 16:
Day 17:
Day 18:
Day 19:
Day 20:
Day 21:
Day 22:
  Italy (home) vs Germany (away)
Day 23:
Day 24:
Day 25:
Day 26:
Day 27:
  Egypt (home) vs Morocco (away)
Day 28:
Day 29:
  Brazil (home) vs Argentina (away)
Day 30:
Day 31:
  Turkey (home) vs New Zealand (away)
Day 32:
Day 33:
  Brazil (home) vs Spain (away)
Day 34:
  Argentina (home) vs Colombia (away)
Day 35:
  Netherlands (home) vs Ecuador (away)
Day 36:
Day 37:
Day 38:
  Colombia (home) vs Brazil (away)
  Turkey (home) vs Egypt (away)
Day 39:
Day 40:
Day 41:
Day 42:
  South Korea (home) vs Mexico (away)
Day 43:
Day 44:
Day 45:
  

In [ ]:
for t in days:
    for i, j in itertools.permutations(teams, 2):
        if solver.Value(x[i, t, j]):
            folium.PolyLine(
                (locations[i], locations[j]), color="blue", weight=2.5, opacity=1
            ).add_to(m)
m
